In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
#Intialize the model
model=Sequential()

In [3]:
# Add Convolution Layer
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation="relu"))

In [4]:
#Add Pooling Layer
model.add(MaxPooling2D(pool_size = (2,2)))

In [5]:
#Add Flattening Layer
model.add(Flatten())

In [6]:
#Add Hidden Layer
model.add(Dense(activation="relu",output_dim=60,init="uniform"))
model.add(Dense(activation="relu",output_dim=80,init="uniform"))
model.add(Dense(activation="relu",output_dim=90,init="uniform"))



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60, kernel_initializer="uniform")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=80, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=90, kernel_initializer="uniform")`
  after removing the cwd from sys.path.


In [7]:
#Add Output layer
model.add(Dense(activation="sigmoid",output_dim=1,init="uniform"))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [8]:
#Compile the model
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./225,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./225)

In [10]:
x_train = train_datagen.flow_from_directory('C:/Users/SUSHMA/Desktop/Intenship Project/DataSets/Train_Dataset',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                     class_mode = 'binary')
x_test = test_datagen.flow_from_directory('C:/Users/SUSHMA/Desktop/Intenship Project/DataSets/Test_Dataset',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')



Found 540 images belonging to 2 classes.
Found 75 images belonging to 2 classes.


In [11]:
print(x_train.class_indices)

{'Cancer': 0, 'NoCancer': 1}


In [12]:
model.save("mymodel.h5")

In [13]:
from keras.models import load_model
import numpy as np
import cv2
model=load_model('mymodel.h5')
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [17]:
from skimage.transform import resize
def detect(frame):
    try:
        img = resize(frame,(64,64))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/25.0
        prediction = model.predict(img)
        print(prediction)
        prediction = model.predict_classes(img)
        print(prediction)
    except AttributeError:
        print("shape not found")


In [18]:
frame=cv2.imread(r"C:\Users\SUSHMA\Desktop\Intenship Project\DataSets\Test_Dataset\Cancer\8863_idx5_x1051_y801_class1.png")

data = detect(frame)

[[0.49845982]]
[[0]]


In [19]:
frame

array([[[243, 241, 241],
        [241, 242, 244],
        [241, 240, 242],
        ...,
        [215, 192, 224],
        [171, 123, 193],
        [178, 140, 200]],

       [[242, 241, 241],
        [243, 241, 243],
        [242, 241, 242],
        ...,
        [187, 148, 203],
        [159, 103, 188],
        [179, 138, 189]],

       [[241, 242, 242],
        [238, 231, 234],
        [241, 239, 243],
        ...,
        [165, 116, 194],
        [173, 125, 204],
        [171, 124, 194]],

       ...,

       [[245, 239, 241],
        [240, 237, 242],
        [234, 233, 241],
        ...,
        [240, 241, 241],
        [244, 240, 242],
        [241, 240, 243]],

       [[236, 237, 239],
        [241, 237, 238],
        [242, 240, 244],
        ...,
        [242, 240, 244],
        [244, 240, 244],
        [245, 245, 243]],

       [[240, 238, 240],
        [240, 238, 242],
        [243, 241, 241],
        ...,
        [242, 241, 242],
        [243, 242, 242],
        [242, 238, 241]]

In [20]:
model.fit_generator(x_train,
                         steps_per_epoch = 250,
                         epochs = 8,
                         validation_data = x_test,
                         validation_steps = 63)

Epoch 1/8
250/250 [==============================] - 85s 341ms/step - loss: 0.5702 - accuracy: 0.7018 - val_loss: 0.7773 - val_accuracy: 0.4267
Epoch 2/8
250/250 [==============================] - 52s 208ms/step - loss: 0.4884 - accuracy: 0.7525 - val_loss: 1.0175 - val_accuracy: 0.3467
Epoch 3/8
250/250 [==============================] - 50s 200ms/step - loss: 0.3924 - accuracy: 0.8106 - val_loss: 0.7877 - val_accuracy: 0.4667
Epoch 4/8
250/250 [==============================] - 48s 191ms/step - loss: 0.3395 - accuracy: 0.8322 - val_loss: 1.1959 - val_accuracy: 0.5333
Epoch 5/8
250/250 [==============================] - 50s 199ms/step - loss: 0.2889 - accuracy: 0.8688 - val_loss: 1.3998 - val_accuracy: 0.5067
Epoch 6/8
250/250 [==============================] - 51s 205ms/step - loss: 0.2688 - accuracy: 0.8690 - val_loss: 0.7796 - val_accuracy: 0.4933
Epoch 7/8
250/250 [==============================] - 46s 183ms/step - loss: 0.2347 - accuracy: 0.8817 - val_loss: 2.5981 - val_accuracy: